# Traning XG boost on the data set

In [15]:
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split , RandomizedSearchCV
from sklearn.metrics import recall_score, make_scorer
from sklearn.metrics import classification_report

In [2]:
data = pd.read_csv(r"E:\codes\AI&ML\project\credit risk scoring system\credit-risk-scoring\lending_club_cleaned_v1.csv")


In [3]:
data.head()

,term,fico_range_low,total_acc,pub_rec,revol_util,annual_inc,int_rate,dti,mort_acc,loan_amnt,...,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,application_type_Joint App,initial_list_status_w
0,60,660.0,18.0,0.0,86.8,40440.0,16.99,15.16,1.0,17775.0,...,0,0,0,0,0,0,0,1,0,0
1,60,740.0,26.0,0.0,103.5,59000.0,15.61,16.74,4.0,29175.0,...,0,0,0,0,0,0,0,1,0,0
2,36,700.0,47.0,0.0,11.4,40000.0,7.90,20.34,1.0,6000.0,...,0,0,0,0,0,1,0,0,0,1
3,60,665.0,26.0,0.0,56.2,70000.0,16.99,23.15,7.0,15600.0,...,0,0,0,0,0,0,0,1,0,0
4,36,700.0,15.0,1.0,67.1,120000.0,14.98,17.88,1.0,10000.0,...,0,0,1,0,0,0,0,1,0,0


In [4]:
x = data.drop(columns=['loan_status'])
y = data['loan_status']

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [6]:
clf = XGBClassifier( eval_metric='logloss', random_state=42)

In [7]:
print(clf.get_params().keys())

dict_keys(['objective', 'base_score', 'booster', 'callbacks', 'colsample_bylevel', 'colsample_bynode', 'colsample_bytree', 'device', 'early_stopping_rounds', 'enable_categorical', 'eval_metric', 'feature_types', 'feature_weights', 'gamma', 'grow_policy', 'importance_type', 'interaction_constraints', 'learning_rate', 'max_bin', 'max_cat_threshold', 'max_cat_to_onehot', 'max_delta_step', 'max_depth', 'max_leaves', 'min_child_weight', 'missing', 'monotone_constraints', 'multi_strategy', 'n_estimators', 'n_jobs', 'num_parallel_tree', 'random_state', 'reg_alpha', 'reg_lambda', 'sampling_method', 'scale_pos_weight', 'subsample', 'tree_method', 'validate_parameters', 'verbosity'])


In [8]:
recall_0 = make_scorer(recall_score, pos_label=0)

In [9]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate' : [0.01,0.05,0.1,0.2],
    'max_depth' : [3,5,8,10],
    'min_child_weight' : [1,5,10],
    'gamma' : [0,0.1,0.2],
    'subsample' : [0.8,0.9,1.0],
    'colsample_bytree' : [0.8,0.9,1.0],
    'scale_pos_weight' : [5],
}
random_search = RandomizedSearchCV(
    estimator=clf,
    param_distributions=param_grid,
    n_iter=15,
    cv=3,
    scoring=recall_0,
    random_state=42,
    verbose=1
)
random_search.fit(x_train, y_train)

Fitting 3 folds for each of 15 candidates, totalling 45 fits


,estimator,"XGBClassifier...ree=None, ...)"
,param_distributions,"{'colsample_bytree': [0.8, 0.9, ...], 'gamma': [0, 0.1, ...], 'learning_rate': [0.01, 0.05, ...], 'max_depth': [3, 5, ...], ...}"
,n_iter,15
,scoring,"make_scorer(r..., pos_label=0)"
,n_jobs,None
,refit,True
,cv,3
,verbose,1
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


In [10]:
result = pd.DataFrame(random_search.cv_results_)
result.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_scale_pos_weight,param_n_estimators,param_min_child_weight,param_max_depth,param_learning_rate,param_gamma,param_colsample_bytree,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,1.575796,0.226083,0.061157,0.009055,0.8,5,300,5,5,0.05,0.1,1.0,"{'subsample': 0.8, 'scale_pos_weight': 5, 'n_e...",0.019634,0.020008,0.018037,0.019226,0.000855,9
1,1.421436,0.039879,0.053840,0.006452,0.8,5,300,10,5,0.01,0.2,1.0,"{'subsample': 0.8, 'scale_pos_weight': 5, 'n_e...",0.000000,0.000000,0.000000,0.000000,0.000000,13
2,1.446715,0.050205,0.060302,0.001336,1.0,5,200,10,10,0.20,0.1,0.8,"{'subsample': 1.0, 'scale_pos_weight': 5, 'n_e...",0.116674,0.120233,0.124566,0.120491,0.003227,2
3,2.318904,0.140360,0.096695,0.011256,0.9,5,300,10,10,0.20,0.2,0.8,"{'subsample': 0.9, 'scale_pos_weight': 5, 'n_e...",0.149084,0.145595,0.146548,0.147075,0.001473,1
4,0.801551,0.024399,0.062649,0.019054,1.0,5,200,10,5,0.10,0.2,0.8,"{'subsample': 1.0, 'scale_pos_weight': 5, 'n_e...",0.020479,0.021980,0.020009,0.020823,0.000840,8


In [11]:
best_param = random_search.best_params_

In [13]:
best_clf = random_search.best_estimator_

In [14]:
y_predict = best_clf.predict(x_test)

In [16]:
print(classification_report(y_test,y_predict))

              precision    recall  f1-score   support

           0       0.66      0.13      0.22      7951
           1       0.85      0.99      0.91     39419

    accuracy                           0.84     47370
   macro avg       0.75      0.56      0.57     47370
weighted avg       0.82      0.84      0.80     47370



up here the there is a misunderstanding happen because our recall for defaulter(0) is 0.19 it means it isn't catching anyone and overfitter with safe peoples(1) becse the default feature of the sg boost work here which make class 1 as the main event and automatically increase the scale_pos_weight here is 5 so increase the priority 5 times than the priorty to class 0 that's why it fails here  

Solution : revese the labels
make class 1 = class 0 and class 0 = class 1 

In [17]:
y_train_rev = 1 - y_train
y_test_rev = 1 - y_test

In [18]:
xgb_clf = XGBClassifier(random_state=42 , eval_metric='logloss')

In [19]:
recall_default = make_scorer(recall_score)

In [21]:
random_search_2 = RandomizedSearchCV(
    estimator=xgb_clf,
    param_distributions=param_grid,
    n_iter=15,
    cv=3,
    scoring=recall_default,
    random_state=42,
    verbose=1
)
random_search_2.fit(x_train, y_train_rev)

Fitting 3 folds for each of 15 candidates, totalling 45 fits


,estimator,"XGBClassifier...ree=None, ...)"
,param_distributions,"{'colsample_bytree': [0.8, 0.9, ...], 'gamma': [0, 0.1, ...], 'learning_rate': [0.01, 0.05, ...], 'max_depth': [3, 5, ...], ...}"
,n_iter,15
,scoring,make_scorer(r...hod='predict')
,n_jobs,None
,refit,True
,cv,3
,verbose,1
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


In [22]:
best_clf_2 = random_search_2.best_estimator_
y_predict_2 = best_clf_2.predict(x_test)

In [23]:
print(classification_report(y_test_rev,y_predict_2))

              precision    recall  f1-score   support

           0       0.92      0.72      0.81     39419
           1       0.34      0.71      0.46      7951

    accuracy                           0.71     47370
   macro avg       0.63      0.71      0.63     47370
weighted avg       0.83      0.71      0.75     47370



In [24]:
import joblib

joblib.dump(best_clf_2, 'xgb_model.pkl')

['xgb_model.pkl']